<a href="https://colab.research.google.com/github/jercash/build_a_star_overcashFINAL/blob/main/build_a_star_workshop_overcash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build-a-Star Workshop
*The most heat you'll ever make*

By Jeremy Overcash :)

In this notebook we will build a star using the Equations of Stellar Structure. This should work for a star of any mass on the main sequence.

Since this is a very difficult task, we will be making some preemptive assumptions. We will assume that the star...

- ...can transport all heat radiatively (convective heat transport is an entirely new beast)

- ...will use purely P-P Chain reactions to generate energy. The CNO cycle will be left out

- ...has a similar material composition and ionization as our Sun

- ...has a similar opacity to the Sun, which will remain constant throughout the star

The first step is to import libraries. We will need numpy, scpipy, and matplot:

In [1]:
import numpy as np
import scipy.constants as scp
import scipy.integrate as integrate
import matplotlib.pyplot as plt

# Variables and Constants

Next, we will define our variables and constants. $M$ can be altered by the viewer to change the mass of the star. Notice how there is only ONE thing the user can change; the goal behind this project is to show how *dependent* on its mass a star really is!

In [2]:
#user-defined variables:
M = 1 * 1.989e30 #kg, mass of the star. coverted from solar masses to kg

#constants:
G = 6.67e-11 #m^3/kg*s^2, gravitational constant
pi = scp.pi #our value of pi. this makes it easier to keep track of
X_fraction = 0.740 #in %, mass fraction of Hydrogen in our Sun
mu = 0.6 #unitless
Msolar = 1.989e30 #kg, mass of our Sun
Rsolar = 690000000 #m, radius of our Sun
opacity = 8 #m^2/kg, similar to the Sun's opacity
Eo = 1.08e-12 #W*m^3/kg^2, constant used in energy generation function
sigma = scp.sigma #W/m^2K^4, Stefan-Boltzmann constant
mh = scp.m_p #kg, mass of proton
k = scp.k #boltzmann constant

# Equations and Functions

We will be using many equations to build our star. Here is a list of their names:

- Hydrostatic Equilibrium

- Equation of State

- Mass Continuity

- Radiative Temperature Gradient

- Solar Luminosity

- Energy Generation

There's a lot here, so I will define them in comments as we go. 

First up, we will define an equation to find the radius of the star from the user-defined mass:

In [3]:
if (M / Msolar) == 1.0: #we want the one we actually know to be true!
  R = Rsolar
else:
  if (M / Msolar) > 1.66: #the rules for radius change as stars get large!
    R = 1.33*Rsolar*(M/Msolar)**0.555
  else:
    R = 1.06*Rsolar*(M/Msolar)**0.945

R = int(R)
#theta = np.linspace(0,R,360)

Now that we have our radius, we can write an equation to find density. Since density changes with radial distance, we must define:

In [4]:
def density(r): 
  return 3*M / (4*pi*r**3)

That wasn't so bad! Now, let's get to the good stuff. We can find the mass internal to a radial distance using the Mass Continuity Equation. This is where we need to start integrating, since this equation traditionally gives us the *change* in mass:

In [5]:
def mass(r): #mass continuity equation
  return 4*pi*r**2*density(r)

#constants for integration:
a = 1 #start. cannot use 0 because we will be dividing by this number
N = 10000000 #number of slicess

def integral_mass(r): #integral of mass continuity equation
  b = r
  h = (b-a) / N 
  x = np.linspace(a,b,N+1,endpoint=True)
  y = mass(x)
  return h * (np.sum(y) - 0.5*(y[0]+y[-1]))

In [14]:
print(integral_mass(R))

3.0534165845211672e+32


Now that we have our mass continuity equation ready, we can set up our equation for Hydrostatic Equilibrium. This is *also* a "change-in" equation, so we need to find the integral as well.

In [6]:
def pressure(r): #hydrostatic equilibrium
  return -1*G*integral_mass(r)[0]/r**2

def integral_pressure(r): #integral of hydrostatic equilibrium
  b = r
  h = (b-a) / N 
  x = np.linspace(a,b,N+1,endpoint=True) #Note the N+1 is because for N trapezoids, you need N+1 points
  y = pressure(x)
  return h * (np.sum(y) - 0.5*(y[0]+y[-1]))

In [18]:
print(integral_pressure(R))

MemoryError: ignored

The next equations we need are our Luminosity equations. Before we can define the change in luminosity, we need to define the equation for core temperature of a star, which writes:

In [7]:
def Tcore(r):
  return mu*mh*pressure(r)/(density(R)*k)

Next, the equation for energy generation:

In [8]:
def E(r): 
  return Eo*density(r)*X_fraction**2*(Tcore(r)/1e6)**4

Now we can define our Luminosity as:

In [ ]:
def luminosity(r):
  return 4*pi*r**2*density(r)*E(r)

def integral_luminosity(r):
  return

Finally, our radiative temperature gradient equation can be defined. This is written as:

In [ ]:
def Trad(r):